In [ ]:
!pip install xlsxwriter -q


마켓컬리 리뷰 수집
+ [x] 🆎 connect : token 서버 접속 후, 변경된 URL로 접속??
+ [x] ✅ moduel/unit test
+ [x] 🆗 API(google Function)

In [ ]:

from datetime import datetime, timedelta
import pandas as pd
import requests
# from google.cloud import bigquery
# import pandas_gbq
import json
import time 


### 🆎 connect

In [ ]:
# URL 예시
main = 'https://www.kurly.com'
product_list = 'https://www.kurly.com/shop/goods/goods_view.php?&goodsno=5139268' # php -> encording 이슈 있음
# review_list  = 'https://www.kurly.com/shop/goods/goods_review_list.php?&goodsno=8844'

token_url = 'https://www.kurly.com/nx/api/session'
review_list = 'https://api.kurly.com/product-review/v2/contents-products/5139268/reviews?onlyImage=false&sortType=RECENTLY&size=10'

In [ ]:
# #검증된 agent에, referer_url을 추가함
# referer_url = 'https://www.kurly.com/'
# user_agent_list = [
#     {'Accept': '*/*', 'Connection': 'keep-alive', 'User-Agent': 'Mozilla/5.0 (X11; Linux i686) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.140 Safari/537.36', 'Accept-Encoding': 'gzip, deflate, br', 'Cache-Control': 'max-age=0', 'DNT': '1', 'accept-language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7,ja;q=0.6', 'origin': referer_url},
#     {'Accept': '*/*', 'Connection': 'keep-alive', 'User-Agent': 'Mozilla/5.0 (X11; Linux i686 on x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36 OPR/55.0.2994.61', 'Accept-Encoding': 'gzip, deflate, br', 'Accept-Language': 'en-US;q=0.5,en;q=0.3', 'Upgrade-Insecure-Requests': '1', 'origin': referer_url, 'Pragma': 'no-cache', 'accept-language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7,ja;q=0.6'},
#     {'Accept': '*/*', 'Connection': 'keep-alive', 'User-Agent': 'Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.80 Safari/537.36 OPR/56.0.3051.104', 'Accept-Encoding': 'gzip, deflate, br', 'Cache-Control': 'max-age=0', 'DNT': '1', 'origin': referer_url, 'accept-language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7,ja;q=0.6'},
#     {'Accept': '*/*', 'Connection': 'keep-alive', 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809 Safari/537.36 OPR/58.0.3135.107', 'Accept-Encoding': 'gzip, deflate, br', 'Accept-Language': 'en-US;q=0.5,en;q=0.3', 'Cache-Control': 'max-age=0', 'DNT': '1', 'Upgrade-Insecure-Requests': '1', 'origin': referer_url, 'accept-language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7,ja;q=0.6'},
#     {'Accept': '*/*', 'Connection': 'keep-alive', 'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36', 'Accept-Encoding': 'gzip, deflate, br', 'Accept-Language': 'en-US;q=0.5,en;q=0.3', 'Cache-Control': 'max-age=0', 'DNT': '1', 'Upgrade-Insecure-Requests': '1', 'origin': referer_url, 'Pragma': 'no-cache', 'accept-language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7,ja;q=0.6'},
#     {'Accept': '*/*', 'Connection': 'keep-alive', 'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.81 Safari/537.36 OPR/55.0.2994.47', 'DNT': '1', 'Upgrade-Insecure-Requests': '1', 'Pragma': 'no-cache', 'Accept-Encoding': 'gzip, deflate, br', 'accept-language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7,ja;q=0.6', 'origin': referer_url},
#     {'Accept': '*/*', 'Connection': 'keep-alive', 'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.110 Safari/537.36', 'Accept-Encoding': 'gzip, deflate, br', 'Cache-Control': 'max-age=0', 'Upgrade-Insecure-Requests': '1', 'origin': referer_url, 'accept-language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7,ja;q=0.6'}
#     ]

# # 임의의 1개 agent를 가져온다
# headers = random.choice(user_agent_list)


In [ ]:
df = pd.DataFrame()

s = requests.Session()
kurly = s.get('https://www.kurly.com/nx/api/session')
token = json.loads(kurly.text)
s.headers.update({'authorization' : 'Bearer ' + token['accessToken']})
s.headers.update({'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36' })

review_list = 'https://api.kurly.com/product-review/v2/contents-products/5052824/reviews?onlyImage=false&sortType=RECENTLY&size=10'
# review_list = 'https://api.kurly.com/board/v1/product-inquiry/content-products/5139268/posts?page=1&per_page=10'

kurly_review = s.get(review_list)

In [ ]:

date_ = json.loads(kurly_review.text)
df_temp1 = pd.DataFrame.from_dict(date_['data'])

# data 변환
df = pd.concat([df, df_temp1], ignore_index=True)

df = df[['ownerName', 'dealProductName', 'contents', 'registeredAt',]]
df.contents = df.contents.str.replace('\n', ' ')
df['registeredAt'] = pd.to_datetime(df['registeredAt'])

In [ ]:
df

,ownerName,dealProductName,contents,registeredAt
0,윤**,[상하목장] 마이리틀 유기농 짜먹는 요거트 플레인,아이가 좋아해요 : ),2023-05-08 12:43:51.202696
1,김**,[상하목장] 마이리틀 유기농 짜먹는 요거트 딸기블루베리,아기가항상 찾는거라 매번쟁여요 신선하고 꼼꼼하게 잘 배송되었어요~,2023-05-08 11:26:39.323722
2,박**,[상하목장] 마이리틀 유기농 짜먹는 요거트 딸기블루베리,아기가 좋아하는 요거트,2023-05-07 21:24:39.034473
3,박**,[상하목장] 마이리틀 유기농 짜먹는 요거트 사과바나나,아기가 좋아해서 자주 구매,2023-05-07 21:24:08.017184
4,장**,[상하목장] 마이리틀 유기농 짜먹는 요거트 딸기블루베리,짜먹는 요거트 맛있네요,2023-05-07 21:01:33.096924
5,윤**,[상하목장] 마이리틀 유기농 짜먹는 요거트 사과바나나,아이 간식으로 먹기 편해요,2023-05-07 21:01:14.501802
6,박**,[상하목장] 마이리틀 유기농 짜먹는 요거트 플레인,아기가 너무 좋아함 ㄹㅎ,2023-05-07 15:04:44.081881
7,박**,[상하목장] 마이리틀 유기농 짜먹는 요거트 플레인,아기가 들고 먹기조은데 대신 토퍼껴야함,2023-05-07 15:03:11.468770
8,최**,[상하목장] 마이리틀 유기농 짜먹는 요거트 사과바나나,간단한 간식으로 좋은,2023-05-07 10:45:23.393165
9,황**,[상하목장] 마이리틀 유기농 짜먹는 요거트 플레인,아기 간식으로 좋네요,2023-05-06 22:44:54.194133


In [ ]:
# 다음 페이지 검색
after_ = date_['meta']['pagination']['after']

review_list = 'https://api.kurly.com/product-review/v2/contents-products/5052824/reviews?onlyImage=false&sortType=RECENTLY&size=10&after=' + str(after_)
kurly_review = s.get(review_list)

In [ ]:
# data 변환
date_ = json.loads(kurly_review.text)
df = pd.DataFrame.from_dict(date_['data'])
df = df[['ownerName', 'dealProductName', 'contents', 'registeredAt',]]
df.contents = df.contents.str.replace('\n', ' ')
df['registeredAt'] = pd.to_datetime(df['registeredAt'])

In [ ]:
df

,ownerName,dealProductName,contents,registeredAt
0,김**,[상하목장] 마이리틀 유기농 짜먹는 요거트 사과바나나,아이가 좋아해서 구매합니다,2023-05-06 20:32:23.179661
1,박**,[상하목장] 마이리틀 유기농 짜먹는 요거트 플레인,아기간식 휴대용으로 좋아요,2023-05-06 15:15:53.709326
2,한**,[상하목장] 마이리틀 유기농 짜먹는 요거트 플레인,아기가 별로 안좋아해오,2023-05-05 22:50:26.171882
3,김**,[상하목장] 마이리틀 유기농 짜먹는 요거트 사과바나나,아이가 좋아해요 달달함,2023-05-05 22:29:42.944868
4,최**,[상하목장] 마이리틀 유기농 짜먹는 요거트 딸기블루베리,아이가 넘 좋아해요,2023-05-05 09:45:36.407792
5,장**,[상하목장] 마이리틀 유기농 짜먹는 요거트 딸기블루베리,우유안먹는 아기라 이걸로 간식합니다,2023-05-05 08:58:54.437523
6,박**,[상하목장] 마이리틀 유기농 짜먹는 요거트 플레인,잘 먹습니다. 재주문해요,2023-05-05 07:59:48.186937
7,이**,[상하목장] 마이리틀 유기농 짜먹는 요거트 딸기블루베리,세일해서 잘샀습니다.아이들이 잘먹어요,2023-05-04 13:53:24.341369
8,구**,[상하목장] 마이리틀 유기농 짜먹는 요거트 사과바나나,뚜껑이있어서 휴대용으로 좋아요,2023-05-04 12:35:51.282932
9,구**,[상하목장] 마이리틀 유기농 짜먹는 요거트 딸기블루베리,아이들 먹기 좋네요,2023-05-04 12:35:33.982924


### ✅ moduel/unit test

In [ ]:
code_list = [
            #  '5000601', '5141795', 
            #  '5139271', '5139272',
            #  '5161550',
            #  '5070678', '5070679', '5125881',
            #  '5139268', '5139269',
            #  '5067217', '5027909', '1000136562', 
            #  '1000122557', '5125877', '5125876', '5141965',#어메이징오트
            #  '5044423', '1000087233', '5125880', # 아몬드브리즈
            #  '5060443', '5060442',  #두유
             ]

In [ ]:
code_list = [
            #  '1000068783', '5001291', # 오틀리
            #   '5025380', '5042070', '5060040'  #대학두유
              '5141965'
             ]

In [ ]:
def kurly_review(code_list):
    # date_start = (datetime.today() + timedelta(hours=9) - timedelta(5)).replace(minute=0, hour=0, second=0, microsecond=0).strftime('%Y-%m-%d')
    # date_end   = (datetime.today() + timedelta(hours=9) - timedelta(1)).replace(minute=0, hour=0, second=0, microsecond=0).strftime('%Y-%m-%d')
    df = pd.DataFrame()

    s = requests.Session()
    kurly = s.get('https://www.kurly.com/nx/api/session')
    token = json.loads(kurly.text)
    s.headers.update({'authorization' : 'Bearer ' + token['accessToken']})
    s.headers.update({'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36' })
    date_gijun = (datetime.today() + timedelta(hours=9) - timedelta(77)).replace(minute=0, hour=0, second=0, microsecond=0)

    for code_ in code_list:
        # 첫 접속
        review_list = 'https://api.kurly.com/product-review/v2/contents-products/'+str(code_)+'/reviews?onlyImage=false&sortType=RECENTLY&size=10'
        kurly_review = s.get(review_list)
        data_ = json.loads(kurly_review.text)
        df_temp1 = pd.DataFrame.from_dict(data_['data'])
        df = pd.concat([df, df_temp1], ignore_index=True)
        df['registeredAt'] = df['registeredAt'].str.slice(start=0, stop=10)
        df['registeredAt'] = pd.to_datetime(df['registeredAt'], format = '%Y-%m-%d' )

        # 2page 접속 이후(loop)
        while date_gijun <= min(df['registeredAt']) :
            after_ = data_['meta']['pagination']['after']
            review_list = 'https://api.kurly.com/product-review/v2/contents-products/'+str(code_)+'/reviews?onlyImage=false&sortType=RECENTLY&size=10&after=' + str(after_)
            kurly_review = s.get(review_list)
            data_ = json.loads(kurly_review.text)
            df_temp2 = pd.DataFrame.from_dict(data_['data'])
            df_temp2['registeredAt'] = df_temp2['registeredAt'].str.slice(start=0, stop=10)
            df_temp2['registeredAt'] = pd.to_datetime(df_temp2['registeredAt'], format = '%Y-%m-%d' )
            df = pd.concat([df, df_temp2], ignore_index=True)

    # df 정리
    df['CHANNEL'] = 'kurly'
    df['SCORE'] = '0'
    df['GUBUN'] = '리뷰'
    df = df[['ownerName', 'CHANNEL', 'dealProductName', 'contents', 'SCORE', 'registeredAt','GUBUN']]
    df.contents = df.contents.str.replace('\n', ' ')
    # df['registeredAt'] = df['registeredAt'].str.slice(start=0, stop=10)
    # df['registeredAt'] = pd.to_datetime(df['registeredAt'], format = '%Y-%m-%d' )
    # df = df[ (date_start <= df['registeredAt'] ) &  (date_end > df['registeredAt'] )].reset_index(drop=True)
    df = df.rename(columns ={'ownerName': "USER", 'dealProductName': "PRODUCT", 'contents': "REVIEW", 'contents': "REVIEW", 'registeredAt' : 'DATE' })
    return df 

In [ ]:
(datetime.today() + timedelta(hours=9) - timedelta(155)).replace(minute=0, hour=0, second=0, microsecond=0)

datetime.datetime(2022, 12, 31, 0, 0)

In [ ]:
def kurly_review_before(code_list):
    df = pd.DataFrame()
    s = requests.Session()
    kurly = s.get('https://www.kurly.com/nx/api/session')
    token = json.loads(kurly.text)
    s.headers.update({'authorization' : 'Bearer ' + token['accessToken']})
    s.headers.update({'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36' })
    date_gijun = (datetime.today() + timedelta(hours=9) - timedelta(155)).replace(minute=0, hour=0, second=0, microsecond=0)

    # 첫 접속
    review_list = 'https://api.kurly.com/product-review/v2/contents-products/'+str(code_list)+'/reviews?onlyImage=false&sortType=RECENTLY&size=10'
    kurly_review = s.get(review_list)
    data_ = json.loads(kurly_review.text)
    df_temp1 = pd.DataFrame.from_dict(data_['data'])
    df = pd.concat([df, df_temp1], ignore_index=True)
    df['registeredAt'] = df['registeredAt'].str.slice(start=0, stop=10)
    df['registeredAt'] = pd.to_datetime(df['registeredAt'], format = '%Y-%m-%d' )

    # 2page 접속 이후(loop)
    while date_gijun <= min(df['registeredAt']) :
        after_ = data_['meta']['pagination']['after']
        review_list = 'https://api.kurly.com/product-review/v2/contents-products/'+str(code_list)+'/reviews?onlyImage=false&sortType=RECENTLY&size=10&after=' + str(after_)
        kurly_review = s.get(review_list)
        data_ = json.loads(kurly_review.text)
        df_temp2 = pd.DataFrame.from_dict(data_['data'])
        df_temp2['registeredAt'] = df_temp2['registeredAt'].str.slice(start=0, stop=10)
        df_temp2['registeredAt'] = pd.to_datetime(df_temp2['registeredAt'], format = '%Y-%m-%d' )
        df = pd.concat([df, df_temp2], ignore_index=True)

    df['CHANNEL'] = 'kurly'
    df['SCORE'] = '0'
    df['GUBUN'] = '리뷰'
    df = df[['ownerName', 'CHANNEL', 'dealProductName', 'contents', 'SCORE', 'registeredAt','GUBUN']]
    df.contents = df.contents.str.replace('\n', ' ')
    df = df.rename(columns ={'ownerName': "USER", 'dealProductName': "PRODUCT", 'contents': "REVIEW", 'contents': "REVIEW", 'registeredAt' : 'DATE' })
    return df

In [ ]:
code_list[0]

'5141965'

In [ ]:
df_product0 = kurly_review_before(code_list[0])

In [ ]:
df_product0.PRODUCT.unique()

array(['[매일] 어메이징 오트 바리스타 950mL'], dtype=object)

In [ ]:
# df_product1 = kurly_review_before(code_list[1])

In [ ]:
# df_product2 = kurly_review_before(code_list[2])

In [ ]:
# df_product3 = kurly_review_before(code_list[3])

In [ ]:
# df_product4 = kurly_review_before(code_list[4])

In [ ]:
# df = pd.concat([df_product0, df_product1, df_product2, df_product3, df_product4],ignore_index=True)

In [ ]:
df_product0.PRODUCT.unique()

array(['[아몬드브리즈] 뉴트리플러스 프로틴 (190mL X 24팩)',
       '[매일] 아몬드 브리즈 아몬드&오트 (190mLX24팩)', '[매일] 아몬드 브리즈 식이섬유 (190mLX24팩)',
       '[매일] 매일두유 검은콩 (190mL X 12팩)', '[매일] 매일두유 초콜릿 (190mL X 12팩)'],
      dtype=object)

In [ ]:
s_dt1= (datetime.today()- timedelta(0)).strftime('%Y%m%d')
excel_name = s_dt1+ '_kurly_sample3.xlsx'
with pd.ExcelWriter(excel_name) as writer:
    # df.to_excel(writer, 'Sheet1')
    df_product0.to_excel(writer, 'Sheet2')
    # df_product3.to_excel(writer, 'Sheet3')
# writer.save()

### 🆗 API(google Function)

In [ ]:
# 트리거 
# https://asia-northeast3-boxwood-chassis-356100.cloudfunctions.net/review_kurly_01

In [ ]:
# requirements.txt
# Function dependencies, for example:
# package>=version

# pandas>=0.25.3
# requests>=2.22.0
# google-cloud-bigquery>=1.18.0
# pandas_gbq>=0.17.9

In [ ]:
# main.py
import pandas as pd
from datetime import datetime, timedelta
import requests
import time 
import json
from google.cloud import bigquery
import pandas_gbq

project = 'boxwood-chassis-356100'
client = bigquery.Client(project)
dataset_id = 'review'
table_id = 'review_all_test'
table_name = dataset_id +'.' +table_id

code_list = [
             '5000601', '5141795', 
             '5139271', '5139272',
             '5161550',
             '5070678', '5070679', '5125881',
             '5139268', '5139269',
             '5067217', '5027909', '1000136562', 
             ]

def kurly_review(code_list):
    date_start = (datetime.today() + timedelta(hours=9) - timedelta(1)).replace(minute=0, hour=0, second=0, microsecond=0).strftime('%Y-%m-%d')
    date_end   = (datetime.today() + timedelta(hours=9) - timedelta(0)).replace(minute=0, hour=0, second=0, microsecond=0).strftime('%Y-%m-%d')
    df = pd.DataFrame()

    s = requests.Session()
    kurly = s.get('https://www.kurly.com/nx/api/session')
    token = json.loads(kurly.text)
    s.headers.update({'authorization' : 'Bearer ' + token['accessToken']})
    s.headers.update({'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36' })

    for code_ in code_list:
        # 첫 접속
        review_list = 'https://api.kurly.com/product-review/v2/contents-products/'+str(code_)+'/reviews?onlyImage=false&sortType=RECENTLY&size=10'
        kurly_review = s.get(review_list)
        data_ = json.loads(kurly_review.text)
        df_temp1 = pd.DataFrame.from_dict(data_['data'])
        df = pd.concat([df, df_temp1], ignore_index=True)

        # 2page 접속
        after_ = data_['meta']['pagination']['after']
        review_list = 'https://api.kurly.com/product-review/v2/contents-products/'+str(code_)+'/reviews?onlyImage=false&sortType=RECENTLY&size=10&after=' + str(after_)
        kurly_review = s.get(review_list)
        data_ = json.loads(kurly_review.text)
        df_temp2 = pd.DataFrame.from_dict(data_['data'])
        df = pd.concat([df, df_temp2], ignore_index=True)

        # 3page 접속
        after_ = data_['meta']['pagination']['after']
        review_list = 'https://api.kurly.com/product-review/v2/contents-products/'+str(code_)+'/reviews?onlyImage=false&sortType=RECENTLY&size=10&after=' + str(after_)
        kurly_review = s.get(review_list)
        data_ = json.loads(kurly_review.text)
        df_temp3 = pd.DataFrame.from_dict(data_['data'])
        df = pd.concat([df, df_temp3], ignore_index=True)

    # df 정리
    df['CHANNEL'] = 'kurly'
    df['SCORE'] = '0'
    df['GUBUN'] = '리뷰'
    df = df[['ownerName', 'CHANNEL', 'dealProductName', 'contents', 'SCORE', 'registeredAt','GUBUN']]
    df.contents = df.contents.str.replace('\n', ' ')
    df['registeredAt'] = df['registeredAt'].str.slice(start=0, stop=10)
    df['registeredAt'] = pd.to_datetime(df['registeredAt'], format = '%Y-%m-%d' )
    df = df[ (date_start <= df['registeredAt'] ) &  (date_end > df['registeredAt'] )].reset_index(drop=True)
    df = df.rename(columns ={'ownerName': "USER", 'dealProductName': "PRODUCT", 'contents': "REVIEW", 'contents': "REVIEW", 'registeredAt' : 'DATE' })
    return df 

def kurly_review_insert(t1):
     df_ = kurly_review(code_list)
     df_['DATE'] = pd.to_datetime(df_['DATE']).dt.normalize()
     df_['DATE'] = pd.to_datetime(df_['DATE'])
     return df_
    #  pandas_gbq.to_gbq(df_, table_name, project_id=project, if_exists = 'append' )
    #  return f'Hello World!'


In [ ]:
t_ = kurly_review(code_list)

In [ ]:
t_

,USER,CHANNEL,PRODUCT,REVIEW,SCORE,DATE,GUBUN
0,이**,kurly,[상하목장] 유기농 요구르트 플레인 400g,요거트 달지 않아서 좋아요,0,2023-05-08,리뷰
1,김**,kurly,[상하목장] 유기농 요구르트 플레인 400g,무난한게 좋아서 잘 사먹여요,0,2023-05-08,리뷰
2,전**,kurly,[상하목장] 유기농 요구르트 플레인 400g,잘 사고 있어요 세일 좋아요,0,2023-05-08,리뷰
3,최**,kurly,[상하목장] 유기농 요구르트 플레인 400g,유통기한도 좋고 여러가지 넣어 먹으면 넘 좋아요!,0,2023-05-08,리뷰
4,황**,kurly,[상하목장] 유기농 요구르트 플레인 400g,울아가가 늘먹던거라 재구매합니다,0,2023-05-08,리뷰
...,...,...,...,...,...,...,...
81,홍**,kurly,[매일] 소화가 잘되는 우유 단백질 930ml,우유 먹고 배 안아퍼서 좋아요. 유통기한 넉넉합니다.,0,2023-05-08,리뷰
82,김**,kurly,[매일] 소화가 잘되는 우유 단백질 930ml,일반 락토프리보다 훨씬 고소해요,0,2023-05-08,리뷰
83,김**,kurly,[매일] 소화가 잘되는 우유 단백질 930ml,달달한 느낌이고 소화도 잘 되고 아주 만족합니다.,0,2023-05-08,리뷰
84,김**,kurly,[매일] 소화가 잘되는 우유 단백질 930ml,소화잘되는 우유에 단백질까지! 믿고 마셔요~,0,2023-05-08,리뷰


In [ ]:
date_start = (datetime.today() + timedelta(hours=9) - timedelta(5)).replace(minute=0, hour=0, second=0, microsecond=0).strftime('%Y-%m-%d')
date_end   = (datetime.today() + timedelta(hours=9) - timedelta(1)).replace(minute=0, hour=0, second=0, microsecond=0).strftime('%Y-%m-%d')
date_start, date_end

('2023-05-04', '2023-05-08')

In [ ]:
# main.py
import pandas as pd
from datetime import datetime, timedelta
import requests
import time 
import json
from google.cloud import bigquery
import pandas_gbq


client = bigquery.Client()
project = 'boxwood-chassis-356100'
dataset_id = 'review'
table_id = 'review_all'
table_ref = client.dataset(dataset_id).table(table_id)
Table = client.get_table(table_ref)  # API request

code_list = [
             ['5008844'  , '[상하목장] 유기농 베이비 요구르트 3종 (85g X 4개)' ],
             ['5000601'   , '[상하목장] 유기농 요구르트 플레인 400g' ],
             ['5004424'  , '[상하목장] 마시는 유기농 요구르트 (100mL X 5개)' ],
             ['5007540'  , '[상하목장] 유기농 우유 & 저지방 우유 750mL 2종' ],
             ['5001691'  , '[상하목장] 유기농 우유 3종' ],
             ['5048852' , '[상하목장] 유기농 우유 900mL' ],
             ['5048853' , '[상하목장] 유기농 우유 저지방 900mL' ],
             ['5034775' , '[상하목장] 유기농 멸균 우유 2종(딸기)' ],
             ['5034934' , '[상하목장] 유기농 멸균 우유 2종(코코아)' ],
             ['5034933' , '[상하목장] 유기농 멸균 우유 2종(바나나)' ],
             ['5059019' , '[상하목장] 유기농 주스 2종' ],
             ['5035445' , '[상하목장] 유기농 저지방 우유 200ml*6개입 (멸균)' ],
             ['5070678' , '[상하목장] 얼려먹는 아이스크림 밀크' ],
             ['5070679' , '[상하목장] 얼려먹는 아이스주스 망고' ],
            #  ['5067217' , '[상하키친] 우유크림 브리또 3종' ],
             ['5027909' , '[상하키친] 정통카레 3종' ],
             ['5036731' , '[상하키친] 슬로우키친 파스타소스 3종' ],
      ]

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.141 Safari/537.36'}

def kurly_review(t1):
    date_start = (datetime.today() + timedelta(hours=9) - timedelta(1)).replace(minute=0, hour=0, second=0, microsecond=0)
    date_end   = (datetime.today() + timedelta(hours=9) - timedelta(0)).replace(minute=0, hour=0, second=0, microsecond=0)
    for goodsCode, item in code_list:
        for p in range(1, 5):
            url  = 'https://www.kurly.com/shop/goods/goods_review_list.php?&goodsno=' + str(goodsCode)  + '&page=' + str(p)
            MKK = requests.post(url, headers=headers).text
            bsObj = BeautifulSoup(MKK, "html.parser")
            count = len(bsObj('div', {'class':'tr_line'}))
            for i in range(count):
                num_ = bsObj('div', {'class':'tr_line'})[i]('td', {'align' : 'center'})[0].text.strip()
                if num_ =='공지':
                    pass
                elif num_ == 'BEST':
                    pass
                else:
                    DATE   = bsObj('div', {'class':'tr_line'})[i]('td', {'class' : 'time'})[0].text.strip()
                    DATE   = datetime.strptime(DATE, '%Y-%m-%d')
                    USER   = bsObj('div', {'class':'tr_line'})[i]('td', {'class' : 'user_grade'})[1].text.strip()
                    temp   = bsObj('div', {'class':'tr_line'})[i]('div', {'class':'inner_review'})[0].text.strip()
                    REVIEW = temp.split('\n\n\n')[-1].replace('\n', ' ')
                    try:
                      PRODUCT = temp.split('\n\n\n')[0].split('\n')[1]
                    except: 
                      PRODUCT = item
                    CHANNEL = 'kurly'
                    if date_start <= DATE < date_end:
                        rows_to_insert = [(USER, CHANNEL, PRODUCT,  REVIEW, '0', DATE.strftime('%Y-%m-%d %H:%M:%S'), goodsCode)]
                        # print(rows_to_insert)
                        client.insert_rows(Table, rows_to_insert)
                    else:
                        pass
    return f'Hello World!'